In [35]:
import pandas as pd
import re
import itertools

In [21]:
# Define import functions
def import_and_clean(path):
    file = pd.read_json(path)
    data = pd.DataFrame(list(itertools.chain(*file['message'].apply(lambda x: x['items']))))
    data['earliest_pub_year'] = data['published'].apply(lambda x: x['date-parts'][0][0])
    data['cleaned_abstract'] = data['abstract'].apply(lambda x: re.sub("<.*?>|\n|Abstract|abstract", "", str(x)))
    return data

In [22]:
climate_articles_2013 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2013.json')
climate_articles_2014 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2014.json')
climate_articles_2015 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2015.json')
climate_articles_2016 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2016.json')
climate_articles_2017 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2017.json')
climate_articles_2018 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2018.json')
climate_articles_2019 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2019.json')
climate_articles_2020 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2020.json')
climate_articles_2021 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2021.json')
climate_articles_2022 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2022.json')
climate_articles_2023 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2023.json')

In [23]:
climate_articles = pd.concat([
    climate_articles_2013,
    climate_articles_2014,
    climate_articles_2015,
    climate_articles_2016,
    climate_articles_2017,
    climate_articles_2018,
    climate_articles_2019,
    climate_articles_2020,
    climate_articles_2021,
    climate_articles_2022,
    climate_articles_2023
])

In [ ]:
climate_articles.shape
# 284,231 articles extracted- still need to clean/ check for potential DOI duplicates

(284231, 56)

In [33]:
climate_articles_unique = climate_articles.drop_duplicates(subset = 'DOI', keep ='first')

In [34]:
climate_articles_unique.shape
# all were unique DOI

(284231, 56)

In [37]:
# Detect english in abstract, also filters missing values
climate_articles_unique['english'] = climate_articles_unique['abstract'].apply(lambda x: detect(str(x)) == 'en')

In [38]:
climate_articles_unique['english']

0        False
1         True
2         True
3        False
4        False
         ...  
42237    False
42238     True
42239    False
42240     True
42241    False
Name: english, Length: 284231, dtype: bool

In [41]:
print(climate_articles_unique[climate_articles_unique['english'] == False]['abstract'])

0                                                      NaN
3                                                      NaN
4                                                      NaN
5                                                      NaN
7                                                      NaN
                               ...                        
42234                                                  NaN
42235                                                  NaN
42237                                                  NaN
42239    <jats:p>Ushbu maqolada biznesning nazariy va a...
42241                                                  NaN
Name: abstract, Length: 180999, dtype: object


In [42]:
climate_articles_unique_english = climate_articles_unique[climate_articles_unique['language'] == 'en']
climate_articles_unique_english.shape
# 103,232 unique, English artiles

(103232, 57)

0          en
1          en
2          en
3          en
4          en
         ... 
103227     en
103228     en
103229    NaN
103230    NaN
103231    NaN
Name: language, Length: 103232, dtype: object

In [43]:
climate_articles_unique_english.to_json('climate_articles_unique_english.json')

ValueError: DataFrame index must be unique for orient='columns'.